### Prepare Unity Catalog AI

 - Initializes the Unity Catalog Function client
 - Implements a utility function to register a Unity Catalog Function

In [3]:
from typing import Callable
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration


CLIENT = UnitycatalogFunctionClient(
    api_client=ApiClient(
        configuration=Configuration(host="http://localhost:8080/api/2.1/unity-catalog")
    )
)
CATALOG = "unity"
SCHEMA = "mcp"


def register(func: Callable[..., any]) -> None:
    CLIENT.create_python_function(
        catalog=CATALOG,
        schema=SCHEMA,
        func=func,
    )

### IP Geolocator Utility Function

 - Uses an IP address to geolocate a location on Earth.

In [9]:
def geolocate_ip(ip: str) -> str:
    """
    Returns a IP geolocation information of the given ip address.

    Args:
        ip: The ip address to geolocate

    Returns:
        JSON which represents the geolocation of the given ip.
    """
    import requests
    return requests.get("http://ip-api.com/json/" + ip).json()


register(geolocate_ip)

### GitHub Repository Search Utility Function

 - Searches GitHub repositories specified by the given keywords.

In [ ]:
def search_github(token: str, keyword: list) -> list:
    """
    Searches the GitHub API for repositories using an input keyword.

    Args:
        token: Your GitHub access token.
        keyword: The keyword list for search.

    Returns:
        A nested list of GitHub repositories returned for the keyword list.
        Each result list contains the repository name, url, and description.
    """
    import time
    from github import Github
    from github.GithubException import RateLimitExceededException
    client = Github(token)
    query = keyword + '+in:readme+in:description'
    results = client.search_repositories(query, 'stars', 'desc')
    results_list = []
    for repo in tqdm(range(0, results.totalCount)):
        try:
            results_list.append([results[repo].name, results[repo].url, results[repo].description])
            time.sleep(2)
        except RateLimitExceededException:
            time.sleep(60)
            results_list.append([results[repo].name, results[repo].url, results[repo].description])
    return results_list